# NLP model using Spacy

# 1. Introduction to Spacy

Link: https://spacy.io/ Here we can go to models and choose the language we would like to use.

In [ ]:
!python -m spacy download es_core_news_sm

2022-09-01 08:23:40.141958: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 8.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import spacy

In [ ]:
nlp = spacy.load("es_core_news_sm") #loading the Spanish language package and storing it in a variable

In [ ]:
nlp

In [ ]:
document = """La situación de la pandemia en el
mundo cada vez se vuelve más compleja. En la actualidad, cada país está afrontando
distintos problemas sociales, sanitarios y económicos, lo que ha llevado a una merma
en la calidad de vida de las personas, principalmente de bajos recursos. Dicho esto, 
es fundamental tener políticas adecuadas para enfrentar la incertidumbre."""

In [ ]:
document_spacy = nlp(document)

In [ ]:
document_spacy

La situación de la pandemia en el
mundo cada vez se vuelve más compleja. En la actualidad, cada país está afrontando
distintos problemas sociales, sanitarios y económicos, lo que ha llevado a una merma
en la calidad de vida de las personas, principalmente de bajos recursos. Dicho esto, 
es fundamental tener políticas adecuadas para enfrentar la incertidumbre.

In [ ]:
type(document_spacy) #a class called 'doc' from the library spacy. This class includes the text and all the results
#that are going to be applied on the text once we call a model, e.g. tokenization

spacy.tokens.doc.Doc

## 2. Text preprocessing


In [ ]:
for token in document_spacy:
  print(token)

La
situación
de
la
pandemia
en
el


mundo
cada
vez
se
vuelve
más
compleja
.
En
la
actualidad
,
cada
país
está
afrontando


distintos
problemas
sociales
,
sanitarios
y
económicos
,
lo
que
ha
llevado
a
una
merma


en
la
calidad
de
vida
de
las
personas
,
principalmente
de
bajos
recursos
.
Dicho
esto
,


es
fundamental
tener
políticas
adecuadas
para
enfrentar
la
incertidumbre
.


Detect and remove stop words. For example, the first paragraph, words such as 'la', 'de' don't give much value. Note that **commas and dots are not stop words**, but they also need to be removed.

In [ ]:
for token in document_spacy:
  print(token.text, '--', token.is_stop, '--', token.is_punct)

La -- True -- False
situación -- False -- False
de -- True -- False
la -- True -- False
pandemia -- False -- False
en -- True -- False
el -- True -- False

 -- False -- False
mundo -- False -- False
cada -- True -- False
vez -- True -- False
se -- True -- False
vuelve -- False -- False
más -- True -- False
compleja -- False -- False
. -- False -- True
En -- True -- False
la -- True -- False
actualidad -- False -- False
, -- False -- True
cada -- True -- False
país -- False -- False
está -- True -- False
afrontando -- False -- False

 -- False -- False
distintos -- False -- False
problemas -- False -- False
sociales -- False -- False
, -- False -- True
sanitarios -- False -- False
y -- True -- False
económicos -- False -- False
, -- False -- True
lo -- True -- False
que -- True -- False
ha -- True -- False
llevado -- False -- False
a -- True -- False
una -- True -- False
merma -- False -- False

 -- False -- False
en -- True -- False
la -- True -- False
calidad -- False -- False
de -- T

In [ ]:
document_spacy_clean = [token for token in document_spacy if not token.is_stop and not token.is_punct]
#just leave the text without the stop words and the puntuation

In [ ]:
document_spacy_clean

[situación,
 pandemia,
 ,
 mundo,
 vuelve,
 compleja,
 actualidad,
 país,
 afrontando,
 ,
 distintos,
 problemas,
 sociales,
 sanitarios,
 económicos,
 llevado,
 merma,
 ,
 calidad,
 vida,
 personas,
 principalmente,
 bajos,
 recursos,
 ,
 fundamental,
 políticas,
 adecuadas,
 enfrentar,
 incertidumbre]

LEMMATIZATION
Good strategy if we want to compare texts and see if they are similar, if they refer to similar issues. We get the original word, the meaning of each word (the lemma). 

In [ ]:
for token in document_spacy_clean:
  print(token.lemma_)

situación
pandemia


mundo
volver
complejo
actualidad
país
afrontar


distinto
problema
social
sanitario
económico
llevar
merma


calidad
vida
persona
principalmente
bajo
recurso


fundamental
política
adecuado
enfrentar
incertidumbre


## 3. Named-entity recognition


In [ ]:
employees = "nombre: Nestor Campos email: nestor@micorreo.com país: Chile nombre: Juana Perez email: juana@micorreo.com país:Argentina nombre: Francisca Leiva email: francisca.leiva@gmail.cl país: Uruguay "

In [ ]:
employees_spacy = nlp(employees)

In [ ]:
for token in employees_spacy:
  if token.like_email: #print if token has similar format to an email (like_number, url, is_digit, is_upper)
    print(token.text)

nestor@micorreo.com
juana@micorreo.com
francisca.leiva@gmail.cl


In [ ]:
print(employees_spacy.ents) #extract entities, but what kind of entities?

(Nestor Campos, Chile, Juana Perez, Argentina, Francisca Leiva, Uruguay)


In [ ]:
for entity in employees_spacy.ents:
  print(entity.text, '--', entity.label_) #the label is what we need to work with in order to get the type of entity

Nestor Campos -- PER
Chile -- LOC
Juana Perez -- PER
Argentina -- LOC
Francisca Leiva -- PER
Uruguay -- LOC


In [ ]:
#RENDERIZATION
from spacy import displacy
displacy.render(employees_spacy, style= "ent", jupyter=True)

## 4. Matching based on patterns


In [ ]:
my_trip = """Hace un par de años visité Cuzco para conocer Machu Picchu.
           También he visitado Argentina un par de veces. 
           La última vez, en 2020, pude visitar Boston.
           """
#There's a common pattern in those three phrases, visitar + place      

In [ ]:
my_trip_spacy = nlp(my_trip)

In [ ]:
from spacy.matcher import Matcher

In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
pattern = [{"LEMMA":"visitar"}, {"POS": "PROPN"}] #POS (position), PROPN (pronoun or noun)

In [ ]:
matcher.add("lugares_visitados", [pattern])

In [ ]:
matches_trip = matcher(my_trip_spacy)
matches_trip #it appears the matching (same numeration), then the initial and the final position of the pattern in the sentence. 

[(16290618549988947147, 5, 7),
 (16290618549988947147, 15, 17),
 (16290618549988947147, 31, 33)]

In [ ]:
for id_value, start, end in matches_trip:
  print(my_trip_spacy[start:end].text)

visité Cuzco
visitado Argentina
visitar Boston


## 5. Similiarity
In Machine Learning and NLP, there are three popular similarity or distance metrics - **Euclidean distance, dot product, and cosine similarity**.
Each word is linked to a vector, so two texts are similar when they have similar numeric values.

In [ ]:
text1= nlp('malo')

In [ ]:
text2= nlp('horrible')

In [ ]:
text1.similarity(text2)
#pay attention to the warning message! Obviously, the words horrible and malo are much more similar than 24%

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """Entry point for launching an IPython kernel.


0.24709503388408582

In [ ]:
!python -m spacy download es_core_news_md #need to load a more complex model with more vectors and combinations

2022-09-01 09:44:55.441656: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.3 MB 1.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [ ]:
nlp2 = spacy.load("es_core_news_md")

In [ ]:
text1= nlp2('malo')

In [ ]:
text2= nlp2('horrible')

In [ ]:
text1.similarity(text2) #now we get better results

0.5889570376208377

In [ ]:
#let's compare some larger snippets of text
revision_1 = nlp2('La comida estaba deliciosa')
revision_2 = nlp2('La comida estaba excelente')
revision_3 = nlp2('No me gustó la comida')
revision_4 = nlp2('No me gustó la experiencia')
revision_5 = nlp2('La comida no estaba buena')

In [ ]:
revision_1.similarity(revision_2)

0.9841328990920907

In [ ]:
revision_3.similarity(revision_4)

0.9809007926808403

In [ ]:
revision_3.similarity(revision_5)

0.6037055715091896